
# Predecir si el cliente subscribirá un deposito

## Problema

Hay un dataset que contiene informacion de una campaña de mercadeo de una institucion bancaria portuguesa. La base de datos se genero por llamadas telefonicas, en ocasiones fue necesario realizar varias llamadas a un mismo cliente. El objetivo de la clasificación es predecir si el cliente suscribirá un depósito a plazo.

**Definición de librerias**

In [1]:
# Carga de datos
from ucimlrepo import fetch_ucirepo

# Tratamiento de datos
import pandas as pd

## Construcción pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

## Construcción modelo
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## Exportar el pipeline
import joblib

**Importación de datos**

In [2]:
# fetch dataset 
bank_marketing = fetch_ucirepo(id=222) 

# data (as pandas dataframes) 
X = bank_marketing.data.features 
y = bank_marketing.data.targets 

**Exploración de datos**

La variable objetivo, es una variable dicotómica, en donde el 'Si' representa el 12% de la base total.

In [3]:
y.value_counts()

y  
no     39922
yes     5289
Name: count, dtype: int64

Se cuenta con 16 variables explicativas y 45211 registros en base. La variable contact y poutcome tienen más del 25% de datos ausentes, mientras que 'job' y 'education', tienen menos del 5% de información ausente. Eliminar duración

In [4]:
X.head()

,age,job,marital,education,default,balance,housing,loan,contact,day_of_week,month,duration,campaign,pdays,previous,poutcome
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,may,261,1,-1,0,NaN
1,44,technician,single,secondary,no,29,yes,no,NaN,5,may,151,1,-1,0,NaN
2,33,entrepreneur,married,secondary,no,2,yes,yes,NaN,5,may,76,1,-1,0,NaN
3,47,blue-collar,married,NaN,no,1506,yes,no,NaN,5,may,92,1,-1,0,NaN
4,33,NaN,single,NaN,no,1,no,no,NaN,5,may,198,1,-1,0,NaN


In [5]:
X.shape

(45211, 16)

In [6]:
X.isnull().sum()

age                0
job              288
marital            0
education       1857
default            0
balance            0
housing            0
loan               0
contact        13020
day_of_week        0
month              0
duration           0
campaign           0
pdays              0
previous           0
poutcome       36959
dtype: int64

La edad promedio de las personas contactadas es de 41 años y su saldo promedio mensual es de 1362. Al 50% de los personas se les ha realizado hasta 2 contactos. Al menos el 75% de los clientes no había sido contactado en una campaña previa.

In [7]:
X.describe()

,age,balance,day_of_week,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


Hay 11 categorías de trabajo, 3 de estado civil y de nivel de educación. Las variables de si tienen crédito en mora, si tienen préstamo de vivienda o préstamo personal, son dicotómicas.

In [8]:
X.describe(include = ['O'])

,job,marital,education,default,housing,loan,contact,month,poutcome
count,44923,45211,43354,45211,45211,45211,32191,45211,8252
unique,11,3,3,2,2,2,2,12,3
top,blue-collar,married,secondary,no,yes,no,cellular,may,failure
freq,9732,27214,23202,44396,25130,37967,29285,13766,4901


**Tratamiento de datos**

Exclusión de variables y de datos ausentes.

Se eliminan las variables de 'contact' y 'poutcome' considerando la cantidad de datos ausentes, tambien se elimina la variable 'duration', ya que después del final de la llamada se conoce 'y' por lo que tiene una dirección directa con 'y'. Adicionalmente, se eliminan los registros con valores ausentes en 'job' y 'education'.

In [9]:
X = X.drop(['contact', 'poutcome', 'duration'], axis=1)
X = X.dropna()
# Guardar los índices de los registros sin valores ausentes en X
indices_validos = X.index

# Filtrar Y usando los mismos índices que X
y = y.loc[indices_validos]

print(X.shape)
print(y.shape)

(43193, 13)
(43193, 1)


In [10]:
y.value_counts()

y  
no     38172
yes     5021
Name: count, dtype: int64

### Modelo

**Pipeline**

In [11]:
### Armamos un Pipeline para no tener que perder sparse cuando usamos onehotencoder

categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(exclude=['object']).columns

preprocesador = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_columns),
    ('cat', OneHotEncoder(sparse_output=True,handle_unknown='ignore'), categorical_columns)
    ])

bank_mkt_pipeline=Pipeline(steps=[('preprocesador', preprocesador), 
                ('modelo', LogisticRegression(max_iter=1000))])

bank_mkt_pipeline

Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  Index(['age', 'balance', 'day_of_week', 'campaign', 'pdays', 'previous'], dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'month'], dtype='object'))])),
                ('modelo', LogisticRegression(max_iter=1000))])

**Dividir en datos de entrenamiento y testeo**

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Entrenar el modelo con el pipeline**

In [13]:
bank_mkt_pipeline.fit(X_train, y_train)

c:\Users\LauraSanchez\miniconda3\envs\proyecto_pandas\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\LauraSanchez\miniconda3\envs\proyecto_pandas\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  Index(['age', 'balance', 'day_of_week', 'campaign', 'pdays', 'previous'], dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'month'], dtype='object'))])),
                ('modelo', LogisticRegression(max_iter=1000))])

**Validación del modelo**

In [14]:
y_pred = bank_mkt_pipeline.predict(X_test)

In [15]:
accuracy_score(y_test, y_pred)

0.8856349114480843

In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          no       0.89      1.00      0.94      7658
         yes       0.44      0.03      0.05       981

    accuracy                           0.89      8639
   macro avg       0.67      0.51      0.50      8639
weighted avg       0.84      0.89      0.84      8639



**Exportar modelo**

In [17]:

joblib.dump(bank_mkt_pipeline, '../Datos/bank_mkt_pipeline.pkl')

['../Datos/bank_mkt_pipeline.pkl']

In [23]:
## Prueba
# primer_registro_dict = X.head(1)
# bank_mkt_pipeline.predict(primer_registro_dict)

array(['no'], dtype=object)